In [1]:
from data_utils import *

# Shuffle carry datasets

* `carry_datasets` is the ordered dataset.


In [2]:
carry_datasets = import_carry_datasets(4, 'add')

In [3]:
for carries in carry_datasets.keys():
    carry_ds_size = carry_datasets[carries]['input'].shape[0]
    randomize = np.arange(carry_ds_size)
    np.random.shuffle(randomize)
    carry_datasets[carries]['input'] = carry_datasets[carries]['input'][randomize]
    carry_datasets[carries]['output'] = carry_datasets[carries]['output'][randomize]

In [4]:
carry_datasets[0]['input'][0]

array([0, 0, 0, 0, 1, 1, 1, 0])

In [5]:
carry_datasets[0]['output'][0]

array([0, 0, 0, 0, 1, 1, 1, 0])

In [6]:
def shuffle_carry_datasets(carry_datasets):
    for carries in carry_datasets.keys():
        carry_ds_size = carry_datasets[carries]['input'].shape[0]
        randomize = np.arange(carry_ds_size)
        np.random.shuffle(randomize)
        carry_datasets[carries]['input'] = carry_datasets[carries]['input'][randomize]
        carry_datasets[carries]['output'] = carry_datasets[carries]['output'][randomize]

In [7]:
shuffle_carry_datasets(carry_datasets)

In [8]:
carry_datasets[0]['input'][0]

array([1, 0, 0, 0, 0, 1, 0, 1])

In [9]:
carry_datasets[0]['output'][0]

array([0, 0, 0, 0, 1, 1, 0, 1])

# Add codes to write shuffled carry_datasets

In [10]:
def save_carry_datasets(carry_datasets, operand_digits, operator):
    save_dir = 'data/{}-bit/{}'.format(operand_digits, operator)
    create_dir(save_dir)

    # Ordered carry datasets
    save_path = '{}/carry_datasets.pickle'.format(save_dir)
    with open(save_path, 'wb') as f:
        pickle.dump(carry_datasets, f)
    print("Saved in '{}'.".format(save_path))

    # Ordered carry datasets
    shuffle_carry_datasets(carry_datasets)
    save_path = '{}/shuffled_carry_datasets.pickle'.format(save_dir)
    with open(save_path, 'wb') as f:
        pickle.dump(carry_datasets, f)
    print("Saved in '{}'.".format(save_path))

# `k*l`-fold cross validation

In [19]:
np_input = np.arange(256*3).reshape((256,3))
np_target = np.arange(256*3).reshape((256,3))

In [22]:
def get_KL_fold_CV_sets(np_input, np_target, i_iteration, n_outer_folds=5, n_inner_folds=5):
    '''
    np_input : numpy.ndarray. shape=(n_examples, dim_input).
    np_target : numpy.ndarray. shape=(n_examples, dim_target).
    i_iteration : The iteration index of cross validation.
     - i_iteration should range from 0 to (n_outer_folds * n_inner_folds - 1)
    '''
    if (i_iteration < 0) or (i_iteration >= n_outer_folds * n_inner_folds):
        raise ValueError("i_iteration should range from 0 to (n_outer_folds * n_inner_folds - 1)")
    
    # Get the indices of outer folds and inner folds.
    i_outer_fold = i_iteration // n_outer_folds
    i_inner_fold = i_iteration % n_outer_folds
    
    ds_size = np_input.shape[0]
    outer_fold_size = ds_size // n_outer_folds
    inner_fold_size = (ds_size - outer_fold_size) // n_inner_folds
    
    # Get the start and end indices of the test set.
    i_start_test = i_outer_fold * outer_fold_size
    if i_outer_fold < n_outer_folds - 1:
        i_end_test = (i_outer_fold + 1) * outer_fold_size
    else:
        # Without this flow control, the remainders are never included in the test set.
        i_end_test = ds_size
        
    # Get the test set.
    input_test = np_input[i_start_test:i_end_test,:]
    target_test = np_target[i_start_test:i_end_test,:]
    
    # Get the inner dataset
    inner_input_dataset = np.concatenate((np_input[:i_start_test,:], np_input[i_end_test:,:]), axis=0)
    inner_target_dataset = np.concatenate((np_target[:i_start_test,:], np_target[i_end_test:,:]), axis=0)
    
    # Get the inner fold size
    inner_ds_size = inner_input_dataset.shape[0]
    inner_fold_size = inner_ds_size // n_inner_folds
    
    # Get the start and end indices of the dev set.
    i_start_dev = i_inner_fold * inner_fold_size
    i_end_dev = (i_inner_fold + 1) * inner_fold_size
    
    # Get the dev set.
    input_dev = inner_input_dataset[i_start_dev:i_end_dev,:]
    target_dev = inner_target_dataset[i_start_dev:i_end_dev,:]
    
    # Get the train set.
    input_train = np.concatenate((inner_input_dataset[:i_start_dev,:], inner_input_dataset[i_end_dev:,:]), axis=0)
    target_train = np.concatenate((inner_target_dataset[:i_start_dev,:], inner_target_dataset[i_end_dev:,:]), axis=0)
    
    return (input_train, input_dev, input_test,
            target_train, target_dev, target_test)

In [24]:
n_outer_folds = 5
n_inner_folds = 5
correctness = True
n_cv_iterations = n_outer_folds * n_inner_folds
for i_iteration in range(n_cv_iterations):
    (input_train, input_dev, input_test,
     target_train, target_dev, target_test) = get_KL_fold_CV_sets(np_input, np_target, i_iteration, n_outer_folds, n_inner_folds)
    correctness = correctness and (np_input.shape[0] == input_train.shape[0] + input_dev.shape[0] + input_test.shape[0])
    correctness = correctness and (np_target.shape[0] == target_train.shape[0] + target_dev.shape[0] + target_test.shape[0])
print("Final correctness: {}".format(correctness))

Final correctness: True


In [25]:
n_outer_folds = 10
n_inner_folds = 10
correctness = True
n_cv_iterations = n_outer_folds * n_inner_folds
for i_iteration in range(n_cv_iterations):
    (input_train, input_dev, input_test,
     target_train, target_dev, target_test) = get_KL_fold_CV_sets(np_input, np_target, i_iteration, n_outer_folds, n_inner_folds)
    correctness = correctness and (np_input.shape[0] == input_train.shape[0] + input_dev.shape[0] + input_test.shape[0])
    correctness = correctness and (np_target.shape[0] == target_train.shape[0] + target_dev.shape[0] + target_test.shape[0])
print("Final correctness: {}".format(correctness))

Final correctness: True


# `k*l`-fold cross validation: with carry datasets

In [58]:
carry_datasets = import_carry_datasets(4, 'add', shuffled=True)

In [59]:
input_train_list = list()
target_train_list = list()
input_dev_list = list()
target_dev_list = list()
input_test_list = list()
target_test_list = list()

i_iteration = 0

for carries in carry_datasets.keys():
    np_input = carry_datasets[carries]['input']
    np_target = carry_datasets[carries]['output']
    
    (input_train, input_dev, input_test,
     target_train, target_dev, target_test) = get_KL_fold_CV_sets(np_input, np_target, i_iteration, n_outer_folds=5, n_inner_folds=5)
    
    input_train_list.append(input_train)
    target_train_list.append(target_train)
    input_dev_list.append(input_dev)
    target_dev_list.append(target_dev)
    input_test_list.append(input_test)
    target_test_list.append(target_test)
    
input_train = np.concatenate(input_train_list, axis=0)
target_train = np.concatenate(target_train_list, axis=0)
input_dev = np.concatenate(input_dev_list, axis=0)
target_dev = np.concatenate(target_dev_list, axis=0)
input_test = np.concatenate(input_test_list, axis=0)
target_test = np.concatenate(target_test_list, axis=0)

In [60]:
print(input_train.shape)
print(target_train.shape)
print(input_dev.shape)
print(target_dev.shape)
print(input_test.shape)
print(target_test.shape)

(168, 8)
(168, 8)
(39, 8)
(39, 8)
(49, 8)
(49, 8)


In [61]:
input_train.shape[0] + input_dev.shape[0] + input_test.shape[0]

256

In [62]:
target_train.shape[0] + target_dev.shape[0] + target_test.shape[0]

256

In [63]:
input_train[0], target_train[0]

(array([0, 0, 1, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 1, 1]))

In [64]:
input_dev[0], target_dev[0]

(array([0, 1, 0, 0, 1, 0, 1, 0]), array([0, 0, 0, 0, 1, 1, 1, 0]))

In [65]:
input_test[0], target_test[0]

(array([1, 0, 1, 0, 0, 1, 0, 1]), array([0, 0, 0, 0, 1, 1, 1, 1]))

# Functionalize

In [70]:
def get_KL_fold_CV_sets_from_carry_datasets(operand_digits, operator, i_iteration, n_outer_folds=5, n_inner_folds=5):
    '''
    Parameters
    -----
    operand_digits : int. The number of each operand digits.
    operator : str. One of 'add', 'subtract', 'multiply', 'divide', 'modulo'
    i_iteration : The iteration index of cross validation.
     - i_iteration should range from 0 to (n_outer_folds * n_inner_folds - 1)
    n_outer_folds : int. The number of outer folds.
    n_inner_folds : int. The number of inner folds.

    Returns
    -----
    (input_train, input_dev, input_test, target_train, target_dev, target_test) :
     - For each element, numpy.ndarray. shape=(n_examples, dim).
    splited_carry_datasets : dict.
     - {n_carries:
            {'input': {'train': numpy.ndarray,
                        'dev': numpy.ndarray,
                        'test': numpy.ndarray},
            'output': {'train': numpy.ndarray,
                        'dev': numpy.ndarray,
                        'test': numpy.ndarray}, ...
            }
        }
     -
    '''
    carry_datasets = import_carry_datasets(operand_digits, operator, shuffled=True)

    input_train_list = list()
    target_train_list = list()
    input_dev_list = list()
    target_dev_list = list()
    input_test_list = list()
    target_test_list = list()

    splited_carry_datasets = dict()

    for carries in carry_datasets.keys():
        np_input = carry_datasets[carries]['input']
        np_target = carry_datasets[carries]['output']

        (input_train, input_dev, input_test,
         target_train, target_dev, target_test) = get_KL_fold_CV_sets(np_input, np_target, i_iteration, n_outer_folds, n_inner_folds)

        input_train_list.append(input_train)
        target_train_list.append(target_train)
        input_dev_list.append(input_dev)
        target_dev_list.append(target_dev)
        input_test_list.append(input_test)
        target_test_list.append(target_test)

        # Initialize a dict for the number of carries
        splited_carry_datasets[carries] = dict()
        splited_carry_datasets[carries]['input'] = dict()
        splited_carry_datasets[carries]['output'] = dict()

        splited_carry_datasets[carries]['input']['train'] = input_train
        splited_carry_datasets[carries]['output']['train'] = target_train
        splited_carry_datasets[carries]['input']['dev'] = input_dev
        splited_carry_datasets[carries]['output']['dev'] = target_dev
        splited_carry_datasets[carries]['input']['test'] = input_test
        splited_carry_datasets[carries]['output']['test'] = target_test

    input_train = np.concatenate(input_train_list, axis=0)
    target_train = np.concatenate(target_train_list, axis=0)
    input_dev = np.concatenate(input_dev_list, axis=0)
    target_dev = np.concatenate(target_dev_list, axis=0)
    input_test = np.concatenate(input_test_list, axis=0)
    target_test = np.concatenate(target_test_list, axis=0)

    return (input_train, input_dev, input_test,
            target_train, target_dev, target_test, 
            splited_carry_datasets)

In [71]:
(input_train, input_dev, input_test, target_train, target_dev, target_test, splited_carry_datasets) = get_KL_fold_CV_sets_from_carry_datasets(4, 'add', 0)

In [72]:
input_train.shape[0] + input_dev.shape[0] + input_test.shape[0]

256

In [73]:
target_train.shape[0] + target_dev.shape[0] + target_test.shape[0]

256

In [74]:
input_train[0], target_train[0]

(array([0, 0, 1, 1, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 1, 1]))

In [75]:
input_dev[0], target_dev[0]

(array([0, 1, 0, 0, 1, 0, 1, 0]), array([0, 0, 0, 0, 1, 1, 1, 0]))

In [76]:
input_test[0], target_test[0]

(array([1, 0, 1, 0, 0, 1, 0, 1]), array([0, 0, 0, 0, 1, 1, 1, 1]))

In [86]:
for carries in splited_carry_datasets.keys():
    print(splited_carry_datasets[carries]['input']['train'].shape, splited_carry_datasets[carries]['input']['dev'].shape, splited_carry_datasets[carries]['input']['test'].shape)

(52, 8) (13, 8) (16, 8)
(36, 8) (8, 8) (10, 8)
(34, 8) (8, 8) (10, 8)
(28, 8) (6, 8) (8, 8)
(18, 8) (4, 8) (5, 8)
